In [ ]:
#import the diffrent data frames for each data stream

In [ ]:
#I have the data in the form of start and end times and sleep level


#I'll make noon to noon graphs
#the height of the graphs will be the sleep stage and color and opacity can be the diffrent streams

#i'll want to get an idea of what days have the most overalpping data
#i'll make a list of dates that I am interested in
#for each day in that list I'll add the name of any of the datastreams that 

#for every day
    #make an empty list
    #for every dataframe
        #make an empty lits
        #append all of the sleep stages for that df for that day
        #append to the list for that day

#start exploring how many days have overlapping data
#see what kind of subsets of data can be used to 
